In [2]:
import pandas as pd

In [3]:
df = pd.read_excel('제10차 산업안전보건 실태조사_raw data_건설업_230102.xlsx')

In [4]:
df.head()

,ID,SQ1,SQ2,SQ3,Q1_1A,Q1_1B,Q1_1C,Q1_2A,Q1_2B,Q1_2C,...,Q32_1,Q33,Q33_1,Q34,Q35,Q36,code5,code2,WT1,WT2
0,1,34177,1,9,2021,1,1,2022,12,31,...,3.0,4,2.0,3,없음,없음,42311,42,21.866142,2.268003
1,2,34176,1,9,2021,1,1,2022,2,31,...,3.0,3,2.0,3,없음,없음,42311,42,21.866142,2.268003
2,3,33633,2,9,2020,6,10,2022,11,23,...,3.0,3,3.0,3,없음,없음,42139,42,12.278481,1.273550
3,4,34193,1,9,2019,11,23,2021,11,22,...,4.0,4,4.0,1,없음,없음,42311,42,21.866142,2.268003
4,5,31801,2,9,2021,5,14,2023,11,30,...,3.0,4,3.0,1,외국인 근로자에 대한 취업 제한이 필요하다,근로자의 안전 의식 수준 향상을 위한 제도 마련이 필요하다,41122,41,6.628571,0.687529


In [5]:
len(df.columns)

200

### 전처리

In [6]:
df.columns

Index(['ID', 'SQ1', 'SQ2', 'SQ3', 'Q1_1A', 'Q1_1B', 'Q1_1C', 'Q1_2A', 'Q1_2B',
       'Q1_2C',
       ...
       'Q32_1', 'Q33', 'Q33_1', 'Q34', 'Q35', 'Q36', 'code5', 'code2', 'WT1',
       'WT2'],
      dtype='object', length=200)

In [7]:
df.isna().sum()

ID         0
SQ1        0
SQ2        0
SQ3        0
Q1_1A      0
        ... 
Q36      169
code5      0
code2      0
WT1        0
WT2        0
Length: 200, dtype: int64

#### 불필요한 변수 제거

In [8]:
df = df.drop(['Q1_5','Q6_1','Q6_2'], axis=1)

In [9]:
df = df.loc[:, ~df.columns.str.contains('Q3') | df.columns.isin(['Q3_1_1', 'Q3_2_1'])]

In [10]:
df = df.loc[:, ~df.columns.str.contains('Q7') | df.columns.isin(['Q7_1', 'Q7_2'])]

In [11]:
df = df.loc[:, ~df.columns.str.contains('Q8|Q10|Q12|Q14|Q15|Q17|Q18|Q19|Q20|Q21|Q23|Q24|Q25|Q26|Q30|Q31|Q32|Q33|Q34')]

In [12]:
df = df.loc[:, ~df.columns.str.contains('Q16') | df.columns.isin(['Q16_1', 'Q16_4', 'Q16_13', 'Q16_14', 'Q16_15'])]

In [13]:
df = df.loc[:, ~df.columns.str.contains('Q28') | df.columns.isin(['Q28_1','Q29_1'])]

In [14]:
df = df.loc[:, ~df.columns.str.contains('Q29')]

In [15]:
df = df.drop(['Q1_4_txt','Q6_2_txt','Q7_1','Q7_2','Q22_txt'], axis=1)

In [16]:
df = df.drop(['code5','code2','WT1','WT2'], axis=1)

#### Q3_1_1 컬럼 특정값 대체

In [17]:
df['Q3_1_1'] = df['Q3_1_1'].fillna(0)
df['Q3_2_1'] = df['Q3_2_1'].fillna(0)

#### target 컬럼 전처리

In [18]:
target_lst = ['Q27_1_1','Q27_1_2','Q27_1_3','Q27_2_1','Q27_2_2','Q27_2_3','Q27_3_1','Q27_3_2','Q27_3_3','Q27_4_1','Q27_4_2','Q27_4_3']

In [19]:
df[target_lst] = df[target_lst].replace(999, 0)

In [20]:
df['target'] = df['Q27_1_1'] + df['Q27_1_2'] + df['Q27_1_3'] + df['Q27_2_1'] + df['Q27_2_2'] + df['Q27_2_3'] + df['Q27_3_1'] + df['Q27_3_2'] + df['Q27_3_3'] + df['Q27_4_1'] + df['Q27_4_2'] + df['Q27_4_3'] 

In [21]:
df = df.drop(columns=target_lst)

In [22]:
df[['target']].value_counts()

target
0         965
1         257
2         117
3          61
4          33
5          24
7          10
6           7
8           6
13          3
9           3
90          3
11          2
12          2
10          2
14          1
17          1
20          1
21          1
30          1
34          1
91          1
dtype: int64

#### 데이터 타입 변환

In [23]:
df = df.astype('string')

In [24]:
df['Q1_3'] = df['Q1_3'].astype('int')
df['Q4_1'] = df['Q4_1'].astype('int')
df['Q4_2'] = df['Q4_2'].astype('int')
df['Q4_3'] = df['Q4_3'].astype('int')
df['Q4_4'] = df['Q4_4'].astype('int')

In [25]:
df['target'] = df['target'].apply(lambda x: '0' if x == '0' else '1')

In [26]:
df['target'].value_counts()

0    965
1    537
Name: target, dtype: int64

### 모델링

#### 피처 엔지니어링

In [27]:
df['Q1_2A'] = df['Q1_2A'].astype('int')
df['Q1_1A'] = df['Q1_1A'].astype('int')

In [28]:
df['duration'] = df['Q1_2A'] - df['Q1_1A']

In [29]:
df['Q4_1'] = df['Q4_1'].astype('int')
df['Q4_2'] = df['Q4_2'].astype('int')

In [30]:
df['55_ratio'] = df['Q4_2'] / df['Q4_1']

In [31]:
df['55_plus'] = df['Q4_2'] + df['Q4_1']

In [32]:
df['55_mul'] = df['Q4_2'] * df['Q4_1']

In [33]:
df = df.drop(['Q3_2_1'], axis=1)

In [34]:
df['foreign_ratio'] = df['Q4_3'] / df['Q4_1']

In [35]:
df['women_ratio'] = df['Q4_4'] / df['Q4_1']

In [36]:
df['Q1_1A'] = df['Q1_1A'].astype('string')
df['Q1_2A'] = df['Q1_2A'].astype('string')

In [37]:
df['year'] = df['Q1_1A'] + df['Q1_2A']

In [38]:
df['syear_type'] = df['Q1_1A'] + df['Q3_1_1']

In [39]:
#df['underprivileged'] = df['Q4_3'] + df['Q4_4'] + df['Q4_2']

In [40]:
#df['underprivileged_ratio'] = (df['Q4_3'] + df['Q4_4'] + df['Q4_2']) / df['Q4_1']

In [41]:
numeric_features = df.dtypes[df.dtypes != "string"].index.tolist()

In [42]:
categorical_features = df.dtypes[df.dtypes == "string"].index.tolist()

In [44]:
len(df.columns)

57

In [43]:
y = df.target
X = df.drop(columns='target')

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.model_selection import KFold
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify = y, random_state = 0)

scores = []
oof_pred = np.zeros(X_test.shape[0])  # OOF 저장
kfold = KFold(n_splits=13, shuffle=True, random_state=0)

for train_index, valid_index in kfold.split(X_train, y_train): 
    train_x, valid_x = X_train.iloc[train_index], X_train.iloc[valid_index]
    train_y, valid_y = y_train.iloc[train_index], y_train.iloc[valid_index]

    model = CatBoostClassifier(cat_features=categorical_features, verbose=True, random_state=0)
    model.fit(train_x, train_y,
              eval_set=[(valid_x,valid_y)],
              early_stopping_rounds=100
             )

    y_pred = model.predict_proba(valid_x)[:, 1]
    auc = roc_auc_score(valid_y, y_pred)
    scores.append(auc)

    # OOF 예측값 저장
    oof_pred += model.predict_proba(X_test)[:, 1] / kfold.get_n_splits()

In [45]:
scores = np.array(scores) 
print("CV scores: ", scores)
print("CV mean = %.2f" % scores.mean(), "with std = %.2f" % scores.std())

CV scores:  [0.79806452 0.79320988 0.78033088 0.79986523 0.83950617 0.78713528
 0.87334218 0.79032258 0.75944056 0.79084967 0.7716129  0.84309896
 0.81406356]
CV mean = 0.80 with std = 0.03


In [46]:
'''# submission 화일 생성
filename = f'catboost_{CATBOOST_VERSION}_{scores.mean():.2f}.csv'
pd.DataFrame({'ID':test_id, 'Salary':oof_pred}).to_csv(filename, index=False)'''

"# submission 화일 생성\nfilename = f'catboost_{CATBOOST_VERSION}_{scores.mean():.2f}.csv'\npd.DataFrame({'ID':test_id, 'Salary':oof_pred}).to_csv(filename, index=False)"

In [47]:
'''from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify = y, random_state = 0)

model = CatBoostClassifier(random_state=42)
model.fit(X_train, y_train)
pred = model.predict_proba(X_test)[:, 1]
score = roc_auc_score(y_test, pred)'''

'from sklearn.model_selection import train_test_split\nfrom sklearn.metrics import roc_auc_score\n\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify = y, random_state = 0)\n\nmodel = CatBoostClassifier(random_state=42)\nmodel.fit(X_train, y_train)\npred = model.predict_proba(X_test)[:, 1]\nscore = roc_auc_score(y_test, pred)'

In [50]:
len(df.columns)

57

In [48]:
importances = model.feature_importances_
feature_importances = list(zip(df.columns, importances))
sorted_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)
for feature, importance in sorted_importances:
    print(f"{feature}: {importance}")

year: 14.032718108136049
Q1_1A: 5.593929196611181
women_ratio: 5.326340085429716
Q11: 5.149423779239863
Q1_2A: 4.9012396905883815
55_ratio: 4.4216774840968505
55_plus: 4.339061483786595
Q3_1_1: 3.6754400830492253
Q4_1: 2.85617729521685
Q4_2: 2.405745071238564
Q16_14: 2.1899347411117946
SQ2: 2.105154932175734
Q4_3: 2.031523822833045
Q2: 1.9918662069701307
Q1_1B: 1.9619780868196592
Q13_1: 1.9375367368304812
Q28_1: 1.882113649773417
Q4_4: 1.8732701471199342
Q22: 1.8064945573123323
Q16_15: 1.6560015257191556
Q16_1: 1.5951008595136031
Q13_19: 1.4772793045664279
Q13_4: 1.2695524339248658
55_mul: 1.2551038918025101
Q1_2B: 1.1516877827099001
Q13_13: 1.1196278606511119
Q13_11: 1.1130348305599331
duration: 1.07732515039364
Q13_10: 1.0492372658711744
Q13_12: 1.0266188312590692
Q13_14: 1.0241271579264641
Q13_20: 1.0187263972387588
Q1_4: 0.97649761654231
Q13_16: 0.9504344529330743
Q16_13: 0.9371162526293572
Q13_7: 0.9178054937586994
Q13_9: 0.8850311104053699
Q13_8: 0.8599448892515603
Q16_4: 0.79349

In [49]:
#df.to_csv('건설업전처리완료(target수정).csv',encoding='cp949', index=False)